## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

In [2]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

D:\py\envs\7105\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
df = pd.read_csv('Wongnai Reviews - Small.csv')

In [4]:
df.tail()

,Review ID,Review
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...


## **Tokenize Words**

In [5]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [8]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [9]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,..."


## **Create Dictionary**

In [10]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [11]:
print(dictionary.token2id.keys())

ษณีย์', 'I', 'Japan', 'Larb', 'you', 'กลิ่นอาย', 'ข้าวปั้น', 'ชาม', 'ช่างคิด', 'ตคอ', 'ตั๊กแตน', 'บมิ', 'บางนา', 'ปุระ', 'ยำสลัด', 'รีรอ', 'ววว', 'วาซาบิ', 'ว้าว', 'สมะ', 'สะดิ้ง', 'หมอน', 'เกะ', 'เมกะ', 'แมลง', 'โซะ', 'โด้', 'โระ', 'โว', 'ไข่ต้ม', 'ดวงใจ', 'ตามตรง', 'กะทิ', 'ข้าวแกง', 'คำถาม', 'คูปอง', 'ง.', 'งอ', 'ตจ', 'ทองพันชั่ง', 'นู่', 'ปุ๊บ', 'พน', 'มะระ', 'ว.', 'สก', 'อ่างทอง', 'เชีบ', 'เอ', 'โคตร', 'CP', 'กฤษณา', 'ครู่เดียว', 'วันดีคืนดี', 'สมาส', 'อด', 'เข็ง', '        ', 'กล้อ', 'ขาดตอน', 'คล่องตัว', 'น์', 'ประกาย', 'สนามบิน', 'สุข', 'สเต๊ก', 'หรูหรา', 'เกวียน', 'เฟ้รนฟ', 'แกงป่า', 'ๆรส', 'Lamb', 'Satay', 'กะหล่ำ', 'กัว', 'กาน', 'ขลุกขลิก', 'จี', 'จึ', 'ช่', 'ซวน', 'ซาน', 'ซู่', 'ดอก', 'ดึ๋ง', 'บลอค', 'บัว', 'ฟั่น', 'รว', 'ราก', 'วช่วน', 'สะเต๊ะ', 'หมา', 'หยาง', 'หูหนู', 'อุดม', 'ฮัว', 'เข่า', 'เฉ่า', 'เฝิ่น', 'เพี้ยน', 'เสฉวน', 'เส้นก๋วยเตี๋ยว', 'แท้ ๆ', 'โจว', 'โร่', 'done', 'medium', 'rare', 'salmon', 'sirloin', 'steak', 'well', 'ขอโทษ', 'ทั้งสอง', 'ทึ่', 'น้ำตะไคร้', 'บ.

In [12]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [14]:
num_topics = 30
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 18.9 s


In [15]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.090943  0.054468       1        1  11.601131
28    -0.081467  0.036441       2        1   7.410949
26    -0.127245 -0.039718       3        1   6.646695
1     -0.063263  0.120358       4        1   5.906221
21    -0.046408  0.045451       5        1   5.807546
14    -0.091455 -0.037875       6        1   5.291104
5     -0.076251 -0.046921       7        1   5.129776
29    -0.073790 -0.050008       8        1   4.884732
20    -0.044998  0.109304       9        1   4.245365
11    -0.054869  0.013506      10        1   3.903931
25    -0.070953 -0.032468      11        1   3.704138
2     -0.031131 -0.061380      12        1   3.606204
27    -0.067625 -0.019633      13        1   3.375252
16    -0.028439 -0.088595      14        1   2.532119
3     -0.007036  0.029547      15        1   2.495988
15     0.016484  0.004160      16        1   2.450409
18    -0.020299 -0.077921      17        1   2.352343
8     -0.003389  0.039237      18        1   2.270807
19     0.054167 -0.038493      19        1   1.970892
6      0.040455 -0.021510      20        1   1.969164
0      0.052471  0.133089      21        1   1.774899
24     0.060401  0.062159      22        1   1.689355
23     0.066607 -0.066297      23        1   1.586104
9      0.017397 -0.072784      24        1   1.579777
13     0.069024  0.078037      25        1   1.386603
17     0.126502  0.081422      26        1   1.160215
12     0.132047 -0.097243      27        1   1.086680
7      0.084301 -0.010025      28        1   1.010817
22     0.129156 -0.066478      29        1   0.688591
4      0.130546  0.020171      30        1   0.482191, topic_info=      Term        Freq       Total Category  logprob  loglift
36      ชา  231.000000  231.000000  Default  30.0000  30.0000
28       -  237.000000  237.000000  Default  29.0000  29.0000
4      กิน  296.000000  296.000000  Default  28.0000  28.0000
438     นม  132.000000  132.000000  Default  27.0000  27.0000
577   เค้ก  114.000000  114.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
13     บาท    1.043065  218.219476  Topic30  -5.0925  -0.0088
175  เนื้อ    0.952937  120.800097  Topic30  -5.1828   0.4922
438     นม    0.952893  132.964969  Topic30  -5.1829   0.3962
58    เมนู    1.024579  270.838566  Topic30  -5.1104  -0.2427
39      ดี    0.979997  323.592061  Topic30  -5.1548  -0.4651

[2185 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
1475      1  0.169745                                                   
1475      8  0.339491                                                   
1475     12  0.169745                                                   
1475     24  0.339491                                                   
3691      4  0.924102                                                   
...     ...       ...                                                ...
3148     28  0.479713  …................................................
2129      7  0.770003                                                  ′
2130      7  0.770001                                                  ″
3653     21  0.638607                                           ♥)......
5749     21  0.638613                                                 ｀)

[5838 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 29, 27, 2, 22, 15, 6, 30, 21, 12, 26, 3, 28, 17, 4, 16, 19, 9, 20, 7, 1, 25, 24, 10, 14, 18, 13, 8, 23, 5])

In [16]:
model.show_topic(1)

[('นั่ง', 0.02012897),
 ('กาแฟ', 0.018716576),
 ('ดี', 0.016384603),
 ('ทาน', 0.014045301),
 ('หวาน', 0.0134324785),
 ('เค้ก', 0.013136307),
 ('ลอง', 0.012442464),
 ('อร่อย', 0.012412482),
 ('ราคา', 0.011059255),
 ('รส', 0.009319485)]

In [17]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [18]:
df.tail()

,Review ID,Review,Review_tokenized,topics,score
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ...",18,0.996965
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย...",1,0.991984
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน...",5,0.989208
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค...",3,0.838256
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,...",8,0.887287
